In [19]:
import pandas as pd
import os, sys, importlib
sys.path.append(os.path.abspath(".."))
from utils import images_clf
import json

## TEST : autoclf

## labels2emb

In [29]:
## labels_text (prompt):
#？
is_default_pic_labels = {
    "1": "the official Airbnb default profile picture, a gray geometric human silhouette",
    "0": "a normal user-uploaded profile picture"
}

# has_person_labels = {
#     "1": "a photo that contains one or more people",
#     "0": "a photo without any people"
# }
# type_labels = {
#     "life": "a person shown in a real-life scene or activity, with visible environment or lifestyle context.",
#     "pro": "a clean portrait or headshot focused mainly on the face, with little or no background information.",
#     "UNK": "no visible person, or not enough information to determine lifestyle vs portrait."
# }

type_labels = {
    "life": 
        "a photo of a person in a visible daily scene or some activities",
    "pro": 
        "a portrait or headshot,focused mainly on the face, with little or no background information.",
    "UNK": 
        "an image without any people or cannot determine whether it is lifestyle or portrait"
}
# quality_labels = {
#     "high": 
#         "a clear, high-quality photo with good lighting and sharp details",
#     "low": 
#         "a low-quality photo with blur, noise, poor lighting or distortion",
#     "UNK": 
#         "quality cannot be determined"
# }
is_smiling_labels = {
    "1": "a person smiling visibly",
    "0": "a person not smiling",
    "UNK": "no person or cannot see their face"
}
sex_labels = {
    "M": "a photo of a man",
    "F": "a photo of a woman",
    "MIX": "a photo with multiple people of mixed gender",
    "UNK": "the gender of the person cannot be determined or no person present"
}
labels_text = {
    # "has_person": has_person_labels,
    "is_default_pic": is_default_pic_labels,
    "type": type_labels,
    # "quality": quality_labels,
    "is_smiling": is_smiling_labels,
    "sex": sex_labels   
}

with open ("labels/labels_text.json","w", encoding='utf-8') as f :
    json.dump(labels_text, f, indent=2)


In [30]:
# embed labels :
import torch
import numpy as np
from transformers import CLIPProcessor, CLIPModel
device='cuda' if torch.cuda.is_available() else "cpu" 
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

d:\miniconda3\envs\airbnb_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [ ]:
# 1) 读取你的 labels JSON
with open("labels/labels_text.json", "r") as f:
    labels_text = json.load(f)

def embed_text_by_clip(text_list):
    """
    text_list: list of strings
    return: np.array of shape (len(text_list), embedding_dim)
    label 被省去，只留下具体描述

    """
    with torch.no_grad():
        inputs = processor(text=text_list, return_tensors="pt", padding=True).to(device)
        text_features = model.get_text_features(**inputs)  # (N, 512)
        text_features = text_features / text_features.norm(dim=-1, keepdim=True)
        return text_features.cpu().numpy().astype("float32")

# 2) 生成 embedding
labels_emb = {}
for category, dic in labels_text.items():
    texts = list(dic.values())  # e.g. ["life prompt", "pro prompt", "UNK prompt"]
    emb = embed_text_by_clip(texts)  # shape = (num_classes, 512)
    labels_emb[category] = emb


# 3) 保存到单个 npz 文件
np.savez("labels/labels_emb.npz", **labels_emb)
print("[SUCCES] Saved text embeddings → labels/labels_emb.npz")


[SUCCES] Saved text embeddings → labels/labels_emb.npz


In [3]:
#检查：
emb = np.load("labels/labels_emb.npz")
type_emb = emb["type"]       # (3, 512)
quality_emb = emb["quality"] # (3, 512)
sex_emb = emb["sex"]         # (4, 512)
print(type_emb.shape)
# 假设有一个 image embedding img_emb (1, 512)

# import numpy as np
# pred_idx = np.argmax(np.dot(img_emb, type_emb.T))
# pred_label = list(labels_text["type"].keys())[pred_idx]
# print(pred_label)


(3, 512)


## PREDICTION

In [32]:
import numpy as np
import json
import os
from tqdm import tqdm
import time
start_time=time.time()


# ------------------------
# 文件路径
# ------------------------
image_emb_path = "embeddings_SAMPLE/emb_SAMPLE.npz"
text_emb_path = "labels/labels_emb.npz"
text_json_path = "labels/labels_text.json"
output_json_path = "annotations_SAMPLE/autoclf_predictions.json"

# ------------------------
# 读取 embeddings
# ------------------------
image_embs = np.load(image_emb_path)  # keys: "host_id.jpg"
text_embs_np = np.load(text_emb_path)
with open(text_json_path, "r") as f:
    labels_text = json.load(f)
default_pic_emb=image_embs["336591839.jpg"]


# ------------------------
# 预测函数
# ------------------------
def is_default_pic(image_emb, default_pic_emb, threshold=0.95):
    """
    image_emb: np.array (512,)
    default_emb: np.array (512,)
    threshold: cosine similarity threshold
    """
    sim = image_emb @ default_pic_emb  # cosine similarity, embeddings 已经 L2-normalized
    return "1" if sim >= threshold else "0"

def zero_shot_predict(img_emb, text_emb_dict):
    """
    img_emb: np.array (512,)
    text_emb_dict: np.array (num_classes, 512)
    """
    sims = img_emb @ text_emb_dict.T        # cosine similarity
    idx = np.argmax(sims)
    return idx, sims


# ------------------------
# 遍历每张图片
# ------------------------
predictions = {}  # 存储结果
for fname in tqdm(image_embs.files, desc='predict on images...'):
    img_emb = image_embs[fname]

    pred = {}
    for category, text_emb_np in text_embs_np.items():
        #对每一个维度进行分类，添加到pred结果中：
        if category == "is_default_pic":
            pred_label = is_default_pic(img_emb, default_pic_emb)
        else:
            idx, sims = zero_shot_predict(img_emb, text_emb_np)
            label_keys = list(labels_text[category].keys())
            pred_label = label_keys[idx]

        pred[category] = pred_label
        
        #全部判断完之后的检查：
        if pred["is_default_pic"]==1:
            pred = { 
                "type": "UNK",
                # "quality": "low",
                "is_smiling": "UNK",
                "sex": "UNK",
                # "has_person": "0",
                "is_default_pic": "1"
                            } 
    predictions[fname] = pred

# ------------------------
# 保存 JSON
# ------------------------
with open(output_json_path, "w") as f:
    json.dump(predictions, f, indent=2)

end_time=time.time()
print(f"✅ Auto predictions on {len(image_embs)} images saved → {output_json_path}: {end_time-start_time:.2f} sec!")
print(predictions)

predict on images...: 100%|██████████| 20/20 [00:00<00:00, 1224.42it/s]

✅ Auto predictions on 20 images saved → annotations_SAMPLE/autoclf_predictions.json: 0.03 sec!
{'102571900.jpg': {'is_default_pic': '0', 'type': 'life', 'is_smiling': '1', 'sex': 'F'}, '106294215.jpg': {'is_default_pic': '0', 'type': 'UNK', 'is_smiling': 'UNK', 'sex': 'UNK'}, '106365215.jpg': {'is_default_pic': '0', 'type': 'life', 'is_smiling': '1', 'sex': 'M'}, '137154154.jpg': {'is_default_pic': '0', 'type': 'life', 'is_smiling': 'UNK', 'sex': 'F'}, '212791574.jpg': {'is_default_pic': '0', 'type': 'UNK', 'is_smiling': '0', 'sex': 'UNK'}, '2379345.jpg': {'is_default_pic': '0', 'type': 'UNK', 'is_smiling': '0', 'sex': 'UNK'}, '24654560.jpg': {'is_default_pic': '0', 'type': 'life', 'is_smiling': 'UNK', 'sex': 'MIX'}, '2798386.jpg': {'is_default_pic': '0', 'type': 'pro', 'is_smiling': '1', 'sex': 'M'}, '28470251.jpg': {'is_default_pic': '0', 'type': 'pro', 'is_smiling': '1', 'sex': 'F'}, '32741638.jpg': {'is_default_pic': '0', 'type': 'UNK', 'is_smiling': '1', 'sex': 'F'}, '336591839.jp

In [10]:
## evaluation of clip:
import json
with open("annotations_SAMPLE\ls_annotations.json", "r", encoding='utf-8')as j:
    annotations=json.load(j)
annot_keys=[f['data']['filename'] for f in annotations]
print(len(annotations))

20


{'102571900.jpg': {'type': 'life', 'quality': 'low', 'is_smiling': '1', 'sex': 'F', 'has_person': '0', 'is_default_pic': '0'}, '106294215.jpg': {'type': 'UNK', 'quality': 'high', 'is_smiling': 'UNK', 'sex': 'UNK', 'has_person': '0', 'is_default_pic': '0'}, '106365215.jpg': {'type': 'life', 'quality': 'low', 'is_smiling': '1', 'sex': 'M', 'has_person': '0', 'is_default_pic': '0'}, '137154154.jpg': {'type': 'life', 'quality': 'high', 'is_smiling': 'UNK', 'sex': 'F', 'has_person': '0', 'is_default_pic': '0'}, '212791574.jpg': {'type': 'UNK', 'quality': 'low', 'is_smiling': '0', 'sex': 'UNK', 'has_person': '1', 'is_default_pic': '0'}, '2379345.jpg': {'type': 'UNK', 'quality': 'low', 'is_smiling': '0', 'sex': 'UNK', 'has_person': '0', 'is_default_pic': '0'}, '24654560.jpg': {'type': 'life', 'quality': 'low', 'is_smiling': 'UNK', 'sex': 'MIX', 'has_person': '1', 'is_default_pic': '0'}, '2798386.jpg': {'type': 'pro', 'quality': 'low', 'is_smiling': '1', 'sex': 'M', 'has_person': '0', 'is_defa

['102571900.jpg', '106294215.jpg', '106365215.jpg', '137154154.jpg', '212791574.jpg', '2379345.jpg', '24654560.jpg', '2798386.jpg', '28470251.jpg', '32741638.jpg', '336591839.jpg', '425502119.jpg', '517697918.jpg', '52438163.jpg', '52801103.jpg', '553099349.jpg', '57226046.jpg', '71320446.jpg', '873444.jpg', '88933385.jpg']
{'336591839.jpg', '88933385.jpg', '2379345.jpg', '102571900.jpg', '57226046.jpg', '212791574.jpg', '28470251.jpg', '71320446.jpg', '106365215.jpg', '106294215.jpg', '52801103.jpg', '425502119.jpg', '873444.jpg', '24654560.jpg', '553099349.jpg', '137154154.jpg', '32741638.jpg', '2798386.jpg', '517697918.jpg', '52438163.jpg'}
20
{'336591839.jpg', '88933385.jpg', '2379345.jpg', '102571900.jpg', '57226046.jpg', '212791574.jpg', '28470251.jpg', '71320446.jpg', '106365215.jpg', '106294215.jpg', '52801103.jpg', '425502119.jpg', '873444.jpg', '24654560.jpg', '553099349.jpg', '137154154.jpg', '32741638.jpg', '2798386.jpg', '517697918.jpg', '52438163.jpg'}
20


## DETECTION:

In [27]:
import os
import json
from ultralytics import YOLO
from PIL import Image
from tqdm import tqdm
import time
# -------------------------------------------------
# 1. 加载 YOLO 模型（最轻量 CPU/GPU 通吃）
# -------------------------------------------------
model = YOLO("yolov8n.pt")

# -------------------------------------------------
# 2. quality 判断（基于最短边 px）
# -------------------------------------------------
def get_quality_label(img_path):
    try:
        img = Image.open(img_path)
        w, h = img.size
        short_edge = min(w, h)

        # 根据 Airbnb 头像特点：200px 以下几乎肯定 low
        if short_edge >= 300:
            return "high"
        else:
            return "low"
    except:
        return "UNK"

# -------------------------------------------------
# 3. YOLO 判断 has_person
# -------------------------------------------------
def detect_has_person(img_path):
    try:
        # results = model(img_path)[0]  # first result
        results = model(img_path, verbose=False)[0]  # 👈 关闭所有日志输出
        for box in results.boxes:
            cls = int(box.cls[0])
            if results.names[cls] == "person":
                return "1"
        return "0"
    except:
        return "UNK"

# -------------------------------------------------
# 4. main pipeline
# -------------------------------------------------
def detect_person_and_quality(
        images_folder="images_SAMPLE",
        save_json="annotations_SAMPLE/detections.json"
    ):
    start_time=time.time()

    os.makedirs(os.path.dirname(save_json), exist_ok=True)
    records = {}

    image_files = [f for f in os.listdir(images_folder) if f.lower().endswith((".jpg",".jpeg",".png"))]

    # print(f"[INFO] Found {len(image_files)} images in {images_folder}")

    for fname in tqdm(image_files, desc="detecting has_person and picture quality..."):
        fpath = os.path.join(images_folder, fname)

        # YOLO: detect person
        has_person = detect_has_person(fpath)

        # quality: size-based
        quality = get_quality_label(fpath)

        records[fname] = {
            "has_person": has_person,
            "quality": quality
        }

        # print(f"[OK] {fname} → has_person={has_person}, quality={quality}")

    # save json
    with open(save_json, "w") as f:
        json.dump(records, f, indent=2)
    end_time=time.time()

    print(f"[SUCCES] Detection results saved to {save_json} :{end_time-start_time:.2f} sec!")
    return records

# # -------------------------------------------------
# # Run
# # -------------------------------------------------
# if __name__ == "__main__":
#     run_yolo_pipeline()


detect_person_and_quality(
        images_folder="images_SAMPLE",
        save_json="annotations_SAMPLE/detections.json"
    )

detecting has_person and picture quality...: 100%|██████████| 20/20 [00:01<00:00, 13.72it/s]

[SUCCES] Detection results saved to annotations_SAMPLE/detections.json :1.47 sec!


{'102571900.jpg': {'has_person': '1', 'quality': 'low'},
 '106294215.jpg': {'has_person': '0', 'quality': 'low'},
 '106365215.jpg': {'has_person': '1', 'quality': 'low'},
 '137154154.jpg': {'has_person': '1', 'quality': 'low'},
 '212791574.jpg': {'has_person': '0', 'quality': 'low'},
 '2379345.jpg': {'has_person': '0', 'quality': 'low'},
 '24654560.jpg': {'has_person': '1', 'quality': 'low'},
 '2798386.jpg': {'has_person': '1', 'quality': 'low'},
 '28470251.jpg': {'has_person': '1', 'quality': 'low'},
 '32741638.jpg': {'has_person': '0', 'quality': 'low'},
 '336591839.jpg': {'has_person': '0', 'quality': 'low'},
 '425502119.jpg': {'has_person': '0', 'quality': 'low'},
 '517697918.jpg': {'has_person': '1', 'quality': 'low'},
 '52438163.jpg': {'has_person': '1', 'quality': 'low'},
 '52801103.jpg': {'has_person': '0', 'quality': 'low'},
 '553099349.jpg': {'has_person': '1', 'quality': 'low'},
 '57226046.jpg': {'has_person': '1', 'quality': 'low'},
 '71320446.jpg': {'has_person': '1', 'qua

## check:

In [ ]:
#检查images, annot, pred 的keys是否对应！！
import os
import json
with open("annotations_SAMPLE/autoclf_predictions.json", "r", encoding='utf-8')as j:
    predictions=json.load(j)
print(predictions)
pred_keys=predictions.keys()
print(len(predictions))


sample_keys=[k for k in os.listdir('images_SAMPLE') if k.endswith('.jpg')]
print(sample_keys)
inter=set(annot_keys).intersection(set(pred_keys))
print(inter)
print(len(inter))


inter=set(sample_keys).intersection(set(pred_keys))
print(inter)
print(len(inter))

## eval:

# APPLICATION: